In [1]:
import pandas as pd
import requests
from lxml import html
from fake_useragent import UserAgent
from time import sleep
from random import randint
from tqdm import tqdm, trange

<b>Постановка задачи</b><br>
Спарсить все рестораны Москвы(данные, отзывы, изображения) с сайта restoran.ru

<i>Тест 1</i> - проверка на чувствительность к парсингу

In [11]:
page = requests.get('https://www.restoran.ru/msk/catalog/restaurants/all/') #первая страница каталога ресторанов

In [17]:
page.status_code

200

In [12]:
data = html.fromstring(page.text)

In [13]:
#Из кода сайта мы знаем, что все рестораны хранятся в блоке <div class="priority-list">
#отдельный ресторан хранится в блоке <div class="item">
#отсюда получаем список ресторанов rest_list
rest_list = data.xpath('//div[@class="priority-list"]/div[@class="item"]') 

In [39]:
#На примере первого ресторана в списке, узнаем какие данные мы получили в явном виде
for item in rest_list[0].text_content().split('\n'):
    if item != '':
        print(item.strip())

КМ
Ресторан
Sixty / Сиксти
Забронировать столик
Смотреть на карте
4.5
262 отзыва
В избранное
Реклама
Кухня:
Средиземноморская, Итальянская, Русская
Средний счет :
3000-4000р
Время работы:
пн-вс с 12:00 до последнего гостя
Телефон:
+7 (495) 988-26-56
Адрес:
Москва, Пресненская наб., д. 12
M
Международная (331 м)
Охраняемая платная


Очевидно, что в явном виде получили поля:
<ul>
<li>Название ресторан</li>
<li>Рейтинг</li>
<li>Количество отзывов</li>
<li>Премиумность ресторана(готов ли владельцы платить нам за потенциальных клиентов)</li>
<li>Типы кухни</li>
<li>Средний счет</li>
<li>Время работы</li>
<li>Номер телефона</li>
<li>Адрес</li>
<li>Тип парковки</li>
</ul>
Неплохо конечно, но хотелось бы и линк на ресторан, и фотографии, и отзывы, да и данных побольше. Как с этим быть?
Помимо явных данных, мы можем вытащить и неявные данные. Вот здесь начинается магия!!!<br>
<i>Неявные данные</i> - данные, передаваемые непосредственно в коде сайта и не предназначенные для отображения конечному пользователю, но видимые для поисковых роботов<s> и программистов</s>.<br>
Чтобы получить неявные данные необходимо взглянуть на код сайта, а конкретно в нашем случае, на код внутри элемента div с class="item"

<i>Номер 1</i>
Сразу получаем Широту, Долготу(lon,lat) и ID ресторана на сайте
![T_start](ws1.png)

<i>Номер 2</i> Получаем ссылку на ресторан внутри сайта restoran.ru и, о боги, полный список изображений ресторана(правда, с не очень подходящим для нас размером)
![T_start](ws2.png)

<i>Номер 3</i> Еще немного потусили в коде и уже получили ссылку на отзывы для ресторана
![T_start](ws3.png)

<i>Номер 4</i> Вот и еще данные - рубрикатор по кухне и метро(здесь важна и транслитная часть названий)
![T_start](ws4.png)

Таким образом, мы получили общую картину происходящего и начальный плацдарм для парсинга. Следующий шаг - определение стратегии парсинга

## Стратегия парсинга

Стратегия парсинга определяется из начальных условий задачи:
<ol>
<li>Время на парсинг</li>
<li>Полнота данных</li>
<li>Качество данных</li>
</ol>
Для данного проекта следующие начальные условия: быстрый парсинг(1-3 дня), самые полные данные, очень чистые и красивые данные.

Взглянув на полученную нами картину, мы можем выбрать оптимальную стратегию парсинга.
### 1. Наивный вариант
Пробегаем все страницы каталога и забираем явные не явные данные, получаем: <i>N страниц каталогов* время на одну страницы (1)</i><br>
Стоп, но ведь нам нужны описания и отзывы о ресторанах?
То есть, нам надо зайти на каждую страницу сайта, забрать с нее описание, а затем зайти на каждую страницу отзывов и забрать все отзывы. Как-то не очень эффективно <i>[(формула 1) + (M страниц ресторанов * время на одну страницу) + (K страниц отзывов * время на одну страниц)]</i>, мягко говоря, да и по времени просядем, а еще и IP поблочат за частые обращения к серверу(DDoS не пройдет).<br> 
К сожалению, в рамках данного проекта лайфхаки и фишечки, ускоряющие процесс парсинга не работают и наивный вариант является единственным верным(стохастический варианты и случайные блуждания мы рассматривать не будем, эти концепции не подходят по начальным условиям нашего проекта).

Очевидно, что парсинг разбивается на 3 этапа:
<ol><li>Парсинг каталога</li><li>Парсинг описаний(+можем допарсить какие-нибудь свистелки/перделки)</li><li>Парсинг отзывов</li></ol>
Опять же, очевидно, что это три разных класса. Причем, классы 2 и 3 зависят только от класса 1, а друг от друга не зависят. Таким образом, первым делом мы будем писать класс-парсер 1, т.е. парсинг каталога.

In [145]:
class Catalog():
    def __init__(self):
        self.res_data = pd.DataFrame()
        #self.t_page = 0
        
    def __getRestList(self, page_num):
        t = randint(2, 6)
        print('Delay %s s.' % (t))
        sleep(t)
        page = requests.get('https://www.restoran.ru/msk/catalog/restaurants/all/?page={}'.format(page_num))
        data = html.fromstring(page.text)
        rest_list = data.xpath('//div[@class="priority-list"]/div[@class="item"]') 
        print('-------------CODE %s-------------' % (page.status_code))
        print('-------------%s rests on page %s-------------' % (len(rest_list), page_num))
        return (rest_list, page.status_code)
    
    def __getProp(self, item, prop_name):
        try:
            props = item.cssselect('div.props div.prop')
        except:
            return '-'

        for prop in props:
            try:#проверка наличия свойств у элемента
                #print(prop.cssselect('div.name')[0].text_content().lower())
                if prop_name.lower() in prop.cssselect('div.name')[0].text_content().lower():
                    return prop.cssselect('div.value')[0].text_content().replace('\n','').strip()
            except:
                return '-'
        else:
            return '-'
    
    def __getMetro(self, item):
        try:
            prop = item.cssselect('div.props div.wsubway')[0]
        except:
            return '-'
        try:
            return prop.cssselect('div.value')[0].text_content().replace('\n','').strip()
        except:
            return '-'
    
    def __getDataFromHTML(self, html_data):
        res = {}
        for item in html_data: #changet to html_data
            ID = item.xpath('div')[0].attrib.get('this-rest-id')
            lon = item.xpath('div')[0].attrib.get('lon')
            lat = item.xpath('div')[0].attrib.get('lat')

            ##########.names-start############
            try:
                rest_type = item.cssselect('div.names div.type')[0].text_content().replace('\n','').strip()
            except:
                rest_type = '-'

            try:
                title = item.cssselect('div.names h2 a')[0].text_content().replace('\n','').strip()
                link = item.cssselect('div.names h2 a')[0].attrib.get('href')
            except:
                title = '-'
                link = '-'

            try:
                rating = item.cssselect('div.names div.list-rating-wrap span')[0].text_content().replace('\n','').strip()
            except:
                rating = 0

            try:
                review_count = item.cssselect('div.names div.list-ration-favorite-wrap div')[1].xpath('a')[0].text_content().replace('\n','').strip()
                review_link = item.cssselect('div.names div.list-ration-favorite-wrap div')[1].xpath('a')[0].attrib.get('href')
            except:
                review_count = 0
                review_link = '-'
            ##########.names-end############

            ##########.props-start############
            try:
                if item.cssselect('div.props div.re')[0].text_content() != '':
                    premium = True
                else:
                    premium = False
            except:
                premium = False
            ##########.props-end############
            
            res[ID] = {
                            'title':title,
                            'link':link,
                            'rating':rating,
                            'review_count':review_count,
                            'review_link':review_link,
                            'lon':lon,
                            'lat':lat,
                            'type':rest_type,
                            'premium':premium,
                            'kitchen':self.__getProp(item, 'кухня'),
                            'avrCheck':self.__getProp(item, 'средний счет'),
                            'timing':self.__getProp(item, 'время работы'),
                            'address':self.__getProp(item, 'адрес'),
                            'phone':self.__getProp(item, 'телефон'),
                            'metro':self.__getMetro(item)
                            }

        return pd.DataFrame.from_dict(res, orient='index')

    def run(self, start_page = 0):
        page = start_page
        status_code = 200
        while status_code == 200 and page < 68:
            print('-------------PAGE %s PARSING-------------' % (page))
            data, status_code = self.__getRestList(page)
            parse_data = self.__getDataFromHTML(data)
            self.res_data = self.res_data.append(parse_data)
            print('-------------%s rests append to frame. Shape of frame: %s @ %s-------------' % (parse_data.shape[0], self.res_data.shape[0], self.res_data.shape[1]))
            page += 1

In [146]:
a = Catalog()

In [148]:
%time a.run()

-------------PAGE 0 PARSING-------------
Delay 5 s.
-------------CODE 200-------------
-------------20 rests on page 0-------------
-------------20 rests append to frame. Shape of frame: 20 @ 15-------------
-------------PAGE 1 PARSING-------------
Delay 2 s.
-------------CODE 200-------------
-------------20 rests on page 1-------------
-------------20 rests append to frame. Shape of frame: 40 @ 15-------------
-------------PAGE 2 PARSING-------------
Delay 2 s.
-------------CODE 200-------------
-------------20 rests on page 2-------------
-------------20 rests append to frame. Shape of frame: 60 @ 15-------------
-------------PAGE 3 PARSING-------------
Delay 5 s.
-------------CODE 200-------------
-------------20 rests on page 3-------------
-------------20 rests append to frame. Shape of frame: 80 @ 15-------------
-------------PAGE 4 PARSING-------------
Delay 4 s.
-------------CODE 200-------------
-------------20 rests on page 4-------------
-------------20 rests append to fram

-------------CODE 200-------------
-------------20 rests on page 39-------------
-------------20 rests append to frame. Shape of frame: 800 @ 15-------------
-------------PAGE 40 PARSING-------------
Delay 5 s.
-------------CODE 200-------------
-------------20 rests on page 40-------------
-------------20 rests append to frame. Shape of frame: 820 @ 15-------------
-------------PAGE 41 PARSING-------------
Delay 6 s.
-------------CODE 200-------------
-------------20 rests on page 41-------------
-------------20 rests append to frame. Shape of frame: 840 @ 15-------------
-------------PAGE 42 PARSING-------------
Delay 4 s.
-------------CODE 200-------------
-------------20 rests on page 42-------------
-------------20 rests append to frame. Shape of frame: 860 @ 15-------------
-------------PAGE 43 PARSING-------------
Delay 6 s.
-------------CODE 200-------------
-------------20 rests on page 43-------------
-------------20 rests append to frame. Shape of frame: 880 @ 15------------

In [149]:
catalog = a.res_data.drop_duplicates()

In [150]:
catalog.head()

,phone,lon,rating,timing,lat,title,review_link,address,review_count,link,premium,type,kitchen,metro,avrCheck
1049250,+7 (495) 988-26-56,37.59573242574,4.5,12:00-00:00,55.744328498233,Эларджи,/msk/opinions/restaurants/elardzhi/,"Москва, Гагаринский переулок, д. 15а",40 отзывов,/msk/detailed/restaurants/elardzhi/,True,Ресторан,"Домашняя, Грузинская",Кропоткинская (490 м),1500-2000р
1216052,+7 (495) 988-26-56,37.62332425168423,0.0,12:00–00:00,55.76196428280367,Mr. Lee / Мистер Ли,/bitrix/components/restoran/favorite.add/ajax.php,"Москва, ул. Кузнецкий Мост, д. 7",В избранное,/msk/detailed/restaurants/mr_lee/,True,"Ресторан, Суши-бар","Авторская, Паназиатская, Японская","Кузнецкий мост (72 м), Лубянка (364 м), Театра...",3000-4000р
1232207,+7 (495) 778-89-94,37.545842860372,4.0,12:00-00:00,55.726364613119,Ласточка,/msk/opinions/restaurants/lastochka/,"Москва, Лужнецкая наб., причал «Лужники ...",16 отзывов,/msk/detailed/restaurants/lastochka/,True,"Ресторан, Банкетный зал, Теплоход","Японская, Итальянская, Русская",Воробьевы горы (1.98 км),2000-3000р
1430018,+7 (495) 988-26-56,0,0.0,-,0,La Marée / Ла Маре,/bitrix/components/restoran/favorite.add/ajax.php,-,В избранное,/msk/detailed/restaurants/la_maree/,True,Ресторан,"Рыбная, Средиземноморская","Чеховская, Улица 1905 года, Смоленская (Арбатс...",3000-4000р
1699769,+7 (495) 988-26-56,37.583187628311,4.0,12:00-00:00 (до последнего гостя),55.757730600603,SELFIE / Селфи,/msk/opinions/restaurants/selfie/,"Москва, Новинский бульвар, 31, второй эт ...",20 отзывов,/msk/detailed/restaurants/selfie/,True,Ресторан,"Авторская, Европейская","Баррикадная (355 м), Краснопресненская (466 м)",1500-2000р


Парсинг страниц-каталогов(на слэнге - парсинг листингов) завершен. Теперь необходимо обработать данные, почистить от мусора и палева и привести данные в красивый вид.<br>
Основной методологией определения мусора являются стат. показатели данных. Определяем стат показатели и чистим от них фрейм. Все очень просто!

In [153]:
catalog.premium.value_counts()

False    1300
True       20
Name: premium, dtype: int64

In [154]:
catalog.review_count.value_counts()

В избранное    499
1 отзыв        211
2 отзыва        80
3 отзыва        60
4 отзыва        50
5 отзывов       39
6 отзывов       31
7 отзывов       25
8 отзывов       18
13 отзывов      18
9 отзывов       17
12 отзывов      16
15 отзывов      15
16 отзывов      14
20 отзывов      13
14 отзывов      13
10 отзывов      11
33 отзыва        9
24 отзыва        9
11 отзывов       8
41 отзыв         7
22 отзыва        7
18 отзывов       7
36 отзывов       6
25 отзывов       6
23 отзыва        5
35 отзывов       5
21 отзыв         5
50 отзывов       5
29 отзывов       5
              ... 
63 отзыва        1
188 отзывов      1
93 отзыва        1
44 отзыва        1
34 отзыва        1
88 отзывов       1
131 отзыв        1
53 отзыва        1
101 отзыв        1
79 отзывов       1
64 отзыва        1
60 отзывов       1
116 отзывов      1
179 отзывов      1
92 отзыва        1
83 отзыва        1
126 отзывов      1
130 отзывов      1
76 отзывов       1
49 отзывов       1
107 отзывов      1
139 отзывов 

In [155]:
catalog['review_count'] = catalog.review_count.apply(lambda x: 0 if x.lower() == 'в избранное' else int(x.split(' ')[0])).astype(int)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [156]:
catalog.review_count.value_counts()

0      499
1      211
2       80
3       60
4       50
5       39
6       31
7       25
8       18
13      18
9       17
12      16
15      15
16      14
14      13
20      13
10      11
33       9
24       9
11       8
41       7
18       7
22       7
25       6
36       6
35       5
29       5
23       5
21       5
50       5
      ... 
131      1
130      1
134      1
203      1
93       1
188      1
145      1
152      1
139      1
49       1
92       1
88       1
247      1
45       1
44       1
60       1
61       1
63       1
64       1
34       1
76       1
77       1
79       1
81       1
83       1
26       1
85       1
53       1
87       1
86       1
Name: review_count, Length: 92, dtype: int64

In [157]:
catalog.review_link.value_counts()

/bitrix/components/restoran/favorite.add/ajax.php             499
/msk/opinions/restaurants/beluga/                               2
/msk/opinions/restaurants/sixty/                                2
/msk/opinions/restaurants/elardzhi/                             2
/msk/opinions/restaurants/chayka_restoran_yakhta/               2
/msk/opinions/restaurants/azerbaydjan/                          2
/msk/opinions/restaurants/lastochka/                            2
/msk/opinions/restaurants/voronezh/                             2
/msk/opinions/restaurants/oblomov/                              2
/msk/opinions/restaurants/whiterabbit/                          2
/msk/opinions/restaurants/selfie/                               2
/msk/opinions/restaurants/the_mad_cook/                         2
/msk/opinions/restaurants/moregril/                             2
/msk/opinions/restaurants/the-waiters/                          2
/msk/opinions/restaurants/black_thai/                           2
/msk/opini

In [158]:
catalog['review_link'] = catalog.review_link.apply(lambda x: '-' if 'bitrix' in x else x)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [159]:
catalog.review_link.value_counts()

-                                                                      499
/msk/opinions/restaurants/nofar/                                         2
/msk/opinions/restaurants/the_mad_cook/                                  2
/msk/opinions/restaurants/sixty/                                         2
/msk/opinions/restaurants/lastochka/                                     2
/msk/opinions/restaurants/selfie/                                        2
/msk/opinions/restaurants/whiterabbit/                                   2
/msk/opinions/restaurants/elardzhi/                                      2
/msk/opinions/restaurants/beluga/                                        2
/msk/opinions/restaurants/azerbaydjan/                                   2
/msk/opinions/restaurants/voronezh/                                      2
/msk/opinions/restaurants/chayka_restoran_yakhta/                        2
/msk/opinions/restaurants/oblomov/                                       2
/msk/opinions/restaurants

In [160]:
catalog = catalog.reset_index()
catalog['ID'] = catalog['index']
del catalog['index']

In [164]:
catalog['premium'] = False

In [ ]:
catalog['']

In [165]:
catalog = catalog.filter(items=['ID', 'title','premium','type', 'kitchen', 'link', 'metro', 'lon','lat','address','phone','rating','timing','review_count','review_link', 'avrCheck'])

In [166]:
catalog = catalog.drop_duplicates()

In [169]:
catalog[catalog.title == 'Бавариус']

,ID,title,premium,type,kitchen,link,metro,lon,lat,address,phone,rating,timing,review_count,review_link,avrCheck
616,386364,Бавариус,False,"Ресторан, Пивной ресторан","Немецкая, Европейская",/msk/detailed/restaurants/bav/,"Маяковская (110 м), Тверская (676 м)",37.59748347138,55.769382848115,"Москва, ул. Тверская 30/2, стр. 1",+7 (495) 699-42-11,3.3,12:00-24:00,12,/msk/opinions/restaurants/bav/,1500-2000р
617,386365,Бавариус,False,"Ресторан, Пивной ресторан","Немецкая, Европейская",/msk/detailed/restaurants/ba/,Фрунзенская (259 м),37.584410349273,55.728107368533,"Москва, Комсомольский пр., д. 21/10",+7 (499) 245-23-95,3.7,12:00-24:00,16,/msk/opinions/restaurants/ba/,1500-2000р


In [170]:
catalog.head()

,ID,title,premium,type,kitchen,link,metro,lon,lat,address,phone,rating,timing,review_count,review_link,avrCheck
0,1049250,Эларджи,False,Ресторан,"Домашняя, Грузинская",/msk/detailed/restaurants/elardzhi/,Кропоткинская (490 м),37.59573242574,55.744328498233,"Москва, Гагаринский переулок, д. 15а",+7 (495) 988-26-56,4.5,12:00-00:00,40,/msk/opinions/restaurants/elardzhi/,1500-2000р
1,1216052,Mr. Lee / Мистер Ли,False,"Ресторан, Суши-бар","Авторская, Паназиатская, Японская",/msk/detailed/restaurants/mr_lee/,"Кузнецкий мост (72 м), Лубянка (364 м), Театра...",37.62332425168423,55.76196428280367,"Москва, ул. Кузнецкий Мост, д. 7",+7 (495) 988-26-56,0.0,12:00–00:00,0,-,3000-4000р
2,1232207,Ласточка,False,"Ресторан, Банкетный зал, Теплоход","Японская, Итальянская, Русская",/msk/detailed/restaurants/lastochka/,Воробьевы горы (1.98 км),37.545842860372,55.726364613119,"Москва, Лужнецкая наб., причал «Лужники ...",+7 (495) 778-89-94,4.0,12:00-00:00,16,/msk/opinions/restaurants/lastochka/,2000-3000р
3,1430018,La Marée / Ла Маре,False,Ресторан,"Рыбная, Средиземноморская",/msk/detailed/restaurants/la_maree/,"Чеховская, Улица 1905 года, Смоленская (Арбатс...",0,0,-,+7 (495) 988-26-56,0.0,-,0,-,3000-4000р
4,1699769,SELFIE / Селфи,False,Ресторан,"Авторская, Европейская",/msk/detailed/restaurants/selfie/,"Баррикадная (355 м), Краснопресненская (466 м)",37.583187628311,55.757730600603,"Москва, Новинский бульвар, 31, второй эт ...",+7 (495) 988-26-56,4.0,12:00-00:00 (до последнего гостя),20,/msk/opinions/restaurants/selfie/,1500-2000р


In [171]:
catalog.to_csv('MskRestCat.csv', index=False, encoding='UTF-8')

Чего не хватает? Описаний и изображений!<br>
Для выгрузки этих данных мы напишем отдельный класс, который пробежит по всем ресторанам и соберет эти данные.

## Часть 2

In [2]:
catalog = pd.read_csv('MskRestCat.csv')

In [32]:
class Page():
    def __init__(self, data):
        self.catalog = data
        self.res_data = pd.DataFrame()
        self.base_url = 'https://www.restoran.ru'
        
    def __getPage(self, link):
        t = randint(2, 4)
        print('Delay %s s.' % (t))
        sleep(t)
        page = requests.get(self.base_url + link)
        data = html.fromstring(page.text)
        return data
    
    def __getImage(self, page):
        try:
            img = page.cssselect('div.carousel-inner div.item')[0].cssselect('div.inline-wrapper a img')[0].attrib.get('src')
            return self.base_url+img
        except:
            return '-'
    
    def __getDescription(self, page):
        try:
            text = page.cssselect('div.description')[0].text_content()
            return text.replace('\xa0',' ').replace('\n','').replace('.','. ').replace('  ',' ').strip()
        except:
            return '-'
    
    def run(self):
        for link in self.catalog.link:
            if link != '-':
                print('-------------PAGE <%s> PARSING-------------' % (link))
                page = self.__getPage(link)
                dscr = self.__getDescription(page)
                img = self.__getImage(page)
                print('-------------|%s|-------------' % (dscr[:15]))
                print('-------------|%s|-------------' % (img[20:]))
                self.res_data = self.res_data.append(pd.DataFrame({
                    'link':[link], 
                    'description':[dscr],
                    'image':[img]
                }))

In [33]:
data = Page(catalog)

In [34]:
data.run()

-------------PAGE </msk/detailed/restaurants/elardzhi/> PARSING-------------
Delay 2 s.
-------------Ресторан «Элард-------------
-------------.ru/upload/resize_cache/iblock/c09/728_400_2/1355315988.jpg-------------
-------------PAGE </msk/detailed/restaurants/mr_lee/> PARSING-------------
Delay 3 s.
-------------Ресторан Mr. Le-------------
-------------.ru/upload/resize_cache/iblock/08f/728_400_2/002fc3.jpg-------------
-------------PAGE </msk/detailed/restaurants/lastochka/> PARSING-------------
Delay 3 s.
-------------Двухпалубный те-------------
-------------.ru/upload/resize_cache/iblock/f62/728_400_2/lastochka_001bb3.jpg-------------
-------------PAGE </msk/detailed/restaurants/la_maree/> PARSING-------------
Delay 2 s.
-------------«Ла Маре» - изв-------------
--------------------------
-------------PAGE </msk/detailed/restaurants/selfie/> PARSING-------------
Delay 2 s.
-------------Панорамный рест-------------
-------------.ru/upload/resize_cache/iblock/b22/728_400_2/mg_9648_

-------------Вы уже были в г-------------
-------------.ru/upload/resize_cache/iblock/00f/728_400_2/1ofitserskoesobranie_akvarium_0011cf.jpg-------------
-------------PAGE </msk/detailed/restaurants/01cafe/> PARSING-------------
Delay 4 s.
-------------Проходя мимо "0-------------
-------------.ru/upload/resize_cache/iblock/e89/728_400_2/a45ap_zal_foto3.jpg-------------
-------------PAGE </msk/detailed/restaurants/barbara_bar/> PARSING-------------
Delay 4 s.
-------------«Барбара бар» —-------------
-------------.ru/upload/resize_cache/iblock/c1d/728_400_2/zal_002_2479d9.jpg-------------
-------------PAGE </msk/detailed/restaurants/backstagerest/> PARSING-------------
Delay 2 s.
-------------Ресторан Backst-------------
-------------.ru/upload/resize_cache/iblock/6ea/728_400_2/backstage_vip_002ab3.jpg-------------
-------------PAGE </msk/detailed/restaurants/bamboobar/> PARSING-------------
Delay 2 s.
-------------Ресторан Bamboo-------------
-------------.ru/upload/resize_cache/ibloc

-------------Это ресторан… н-------------
-------------.ru/upload/resize_cache/iblock/85d/728_400_2/1_glavnaya_2485.jpg-------------
-------------PAGE </msk/detailed/restaurants/beermarket/> PARSING-------------
Delay 4 s.
-------------Визитными карто-------------
-------------.ru/upload/resize_cache/iblock/2e5/728_400_2/vipzal_00169c.jpg-------------
-------------PAGE </msk/detailed/restaurants/bistrot/> PARSING-------------
Delay 4 s.
-------------Это тот самый б-------------
-------------.ru/upload/resize_cache/iblock/605/728_400_2/e9b9bistrot_1.jpg-------------
-------------PAGE </msk/detailed/restaurants/brasseriemost/> PARSING-------------
Delay 4 s.
-------------Ресторан «Brass-------------
-------------.ru/upload/resize_cache/iblock/21a/728_400_2/img_2014_1d32.jpg-------------
-------------PAGE </msk/detailed/restaurants/stkrolik/> PARSING-------------
Delay 4 s.
---------------------------
-------------.ru/upload/resize_cache/iblock/976/728_400_2/2017_09_26_212523_247e.jpg----

-------------Ресторан Centra-------------
-------------.ru/upload/resize_cache/iblock/f85/728_400_2/k97a8490_23df.jpg-------------
-------------PAGE </msk/detailed/restaurants/casadimosca/> PARSING-------------
Delay 2 s.
-------------Ресторан Casa d-------------
-------------.ru/upload/resize_cache/iblock/edc/728_400_2/casaaef.jpg-------------
-------------PAGE </msk/detailed/restaurants/catch-bistro/> PARSING-------------
Delay 3 s.
-------------Catch Human fri-------------
-------------.ru/upload/resize_cache/iblock/1f9/728_400_2/img_7431_27ee.jpg-------------
-------------PAGE </msk/detailed/restaurants/cantinetta_antinori/> PARSING-------------
Delay 4 s.
-------------Ресторан Cantin-------------
-------------.ru/upload/resize_cache/iblock/4f3/728_400_2/04791e.jpg-------------
-------------PAGE </msk/detailed/restaurants/chelsea-pub/> PARSING-------------
Delay 4 s.
-------------Если вы до сих -------------
-------------.ru/upload/resize_cache/iblock/1b2/728_400_2/1360672650.jpg--

---------------------------
-------------.ru/upload/resize_cache/iblock/88b/728_400_2/2a1a_mg_3547.jpg-------------
-------------PAGE </msk/detailed/restaurants/city-space/> PARSING-------------
Delay 3 s.
-------------Бар-ресторан «С-------------
-------------.ru/upload/resize_cache/iblock/db3/728_400_2/img_2652_23e9.jpg-------------
-------------PAGE </msk/detailed/restaurants/craft-kitchen/> PARSING-------------
Delay 2 s.
---------------------------
-------------.ru/upload/resize_cache/iblock/7d8/728_400_2/craftkitchen_zal_013b65.jpg-------------
-------------PAGE </msk/detailed/restaurants/crystal/> PARSING-------------
Delay 4 s.
-------------Караоке Crystal-------------
-------------.ru/upload/resize_cache/iblock/0ff/728_400_2/a02d7878_2fda.jpg-------------
-------------PAGE </msk/detailed/restaurants/corner/> PARSING-------------
Delay 2 s.
-------------Кафе Corner в М-------------
-------------.ru/upload/resize_cache/iblock/0ae/728_400_2/cornercafekitchen_36_of_152_2060.jpg---

...

-------------Ресторан«Старый-------------
-------------.ru/upload/resize_cache/iblock/396/728_400_2/dsc_0113_2b77.jpg-------------
-------------PAGE </msk/detailed/restaurants/strelets/> PARSING-------------
Delay 4 s.
---------------------------
-------------.ru/upload/resize_cache/iblock/bb6/728_400_2/0d8e698b318d3627dbd8168a5e44a7ad_24cf.jpg-------------
-------------PAGE </msk/detailed/restaurants/stop-snyato/> PARSING-------------
Delay 3 s.
---------------------------
-------------.ru/upload/resize_cache/iblock/213/728_400_2/pict_20160712_22263451c.jpg-------------
-------------PAGE </msk/detailed/restaurants/staryy-baku/> PARSING-------------
Delay 3 s.
---------------------------
-------------.ru/upload/resize_cache/iblock/797/728_400_2/05_2bab.jpg-------------
-------------PAGE </msk/detailed/restaurants/usadbarest/> PARSING-------------
Delay 4 s.
-------------В Перово, вдали-------------
-------------.ru/upload/iblock/d5c/1d2bvid_zala3.jpg-------------
-------------PAGE </ms

---------------------------
-------------.ru/upload/resize_cache/iblock/1e0/728_400_2/img_1135683.jpg-------------
-------------PAGE </msk/detailed/restaurants/treli/> PARSING-------------
Delay 3 s.
---------------------------
-------------.ru/upload/resize_cache/iblock/111/728_400_2/71be6.jpg-------------
-------------PAGE </msk/detailed/restaurants/trattoria-di-djulia/> PARSING-------------
Delay 2 s.
-------------Траттория «Ди Д-------------
-------------.ru/upload/resize_cache/iblock/2ec/728_400_2/img_0885572.jpg-------------
-------------PAGE </msk/detailed/restaurants/traktir-pozharskogo/> PARSING-------------
Delay 4 s.
-------------Ресторан«Тракти-------------
-------------.ru/upload/resize_cache/iblock/52c/728_400_2/02606.jpg-------------
-------------PAGE </msk/detailed/restaurants/traktir-novoslobodskaya/> PARSING-------------
Delay 3 s.
---------------------------
-------------.ru/upload/resize_cache/iblock/44f/728_400_2/dsc_0403_22ea.jpg-------------
-------------PAGE </m

---------------------------
-------------.ru/upload/resize_cache/iblock/afe/728_400_2/dsc_0861_22de.jpg-------------
-------------PAGE </msk/detailed/restaurants/hachmaz/> PARSING-------------
Delay 3 s.
---------------------------
-------------.ru/upload/resize_cache/iblock/f12/728_400_2/dsc_1170_216e.jpg-------------
-------------PAGE </msk/detailed/restaurants/hachapuriya/> PARSING-------------
Delay 4 s.
-------------Кафе «Хачапурия-------------
-------------.ru/upload/resize_cache/iblock/e31/728_400_2/ekik_e_gheog35_rp68_jg_article79f.png-------------
-------------PAGE </msk/detailed/restaurants/fortepyano/> PARSING-------------
Delay 2 s.
---------------------------
-------------.ru/upload/resize_cache/iblock/9a3/728_400_2/marifg0mezq_2d65.jpg-------------
-------------PAGE </msk/detailed/restaurants/uyutnoe-mesto/> PARSING-------------
Delay 3 s.
-------------Кафе «Уютное ме-------------
-------------.ru/upload/resize_cache/iblock/8c1/728_400_2/img_1225_29_30_fused2_2cf1.jpg----

-------------В ресторанах «Т-------------
--------------------------
-------------PAGE </msk/detailed/restaurants/pavlinmavlin/> PARSING-------------
Delay 4 s.
-------------Шесть ресторано-------------
--------------------------
-------------PAGE </msk/detailed/restaurants/pavlin_mavlin_bakuninskaya/> PARSING-------------
Delay 2 s.
-------------Ресторан «Чайха-------------
-------------.ru/upload/resize_cache/iblock/ec3/728_400_2/bakuninskaya_zal1_001bf1.jpg-------------
-------------PAGE </msk/detailed/restaurants/pavlin_mavlin_koroleva/> PARSING-------------
Delay 3 s.
-------------Ресторан «Чайха-------------
-------------.ru/upload/resize_cache/iblock/26b/728_400_2/akkoroleva_vipzal_002dcb.jpg-------------
-------------PAGE </msk/detailed/restaurants/pavlin_mavlin_profsoyuznaya/> PARSING-------------
Delay 2 s.
-------------Ресторан «Чайха-------------
-------------.ru/upload/resize_cache/iblock/e8c/728_400_2/profsoyuznaya_bar_001ac0.jpg-------------
-------------PAGE </msk/detai

---------------------------
-------------.ru/upload/resize_cache/iblock/c92/728_400_2/p3231211_23ac.jpg-------------
-------------PAGE </msk/detailed/restaurants/expromt_bar/> PARSING-------------
Delay 2 s.
-------------Ресторан «Экспр-------------
-------------.ru/upload/resize_cache/iblock/582/728_400_2/img_8009f07.jpg-------------
-------------PAGE </msk/detailed/restaurants/yugoslaviya/> PARSING-------------
Delay 2 s.
-------------Ресторан «Югосл-------------
-------------.ru/upload/resize_cache/iblock/f02/728_400_2/img_3651_edit_edit_2f12.jpg-------------
-------------PAGE </msk/detailed/restaurants/ekspeditsiya_expedition/> PARSING-------------
Delay 2 s.
-------------Позади сотни ма-------------
-------------.ru/upload/resize_cache/iblock/2d2/728_400_2/img_82612aa.jpg-------------
-------------PAGE </msk/detailed/restaurants/shisha/> PARSING-------------
Delay 2 s.
---------------------------
-------------.ru/upload/resize_cache/iblock/bd9/728_400_2/a827b4_d74ef95c607f47068666

In [36]:
data.res_data.to_csv('backup.csv', index=False, encoding='utf-8')

In [58]:
pages = data.res_data.drop_duplicates()

In [59]:
pages.shape

(1300, 3)

In [82]:
t = catalog.merge(pages, on='link').drop_duplicates()

In [63]:
t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1306 entries, 0 to 1305
Data columns (total 18 columns):
ID              1306 non-null int64
title           1306 non-null object
premium         1306 non-null bool
type            1299 non-null object
kitchen         1306 non-null object
link            1306 non-null object
metro           1306 non-null object
lon             1296 non-null float64
lat             1296 non-null float64
address         1306 non-null object
phone           1306 non-null object
rating          1306 non-null float64
timing          1306 non-null object
review_count    1306 non-null int64
review_link     1306 non-null object
avrCheck        1306 non-null object
description     1306 non-null object
image           1306 non-null object
dtypes: bool(1), float64(3), int64(2), object(12)
memory usage: 184.9+ KB


In [68]:
t[t.link == '/msk/detailed/restaurants/azerbaydjan/']

,ID,title,premium,type,kitchen,link,metro,lon,lat,address,phone,rating,timing,review_count,review_link,avrCheck,description,image
23,388296,Азербайджан,False,"Ресторан, Банкетный зал","Азербайджанская, Турецкая, Восточная",/msk/detailed/restaurants/azerbaydjan/,"Октябрьское поле (2.65 км), Полежаевская (2.18...",37.48683,55.769933,"Москва, ул. Демьяна Бедного, д. 4",+7 (499) 946-34-30 ...,4.0,Круглосуточно,8,/msk/opinions/restaurants/azerbaydjan/,1000-1500р.,Путешествие в настоящую азербайджанскую кухню ...,https://www.restoran.ru/upload/resize_cache/ib...
24,388296,Азербайджан,False,"Ресторан, Банкетный зал","Азербайджанская, Турецкая, Восточная",/msk/detailed/restaurants/azerbaydjan/,"Октябрьское поле (2.65 км), Полежаевская (2.18...",37.48683,55.769933,"Москва, ул. Демьяна Бедного, д. 4",+7 (499) 946-34-30 ...,4.0,Круглосуточно,8,/msk/opinions/restaurants/azerbaydjan/,1000-1500р.,Путешествие в настоящую азербайджанскую кухню ...,https://www.restoran.ru/upload/resize_cache/ib...


In [89]:
for clmn in t.columns.values:
    print(clmn,':', t[t.link == '/msk/detailed/restaurants/lastochka/'][clmn].unique())

ID : [1232207]
title : ['Ласточка']
premium : [False]
type : ['Ресторан, Банкетный зал, Теплоход']
kitchen : ['Японская, Итальянская, Русская']
link : ['/msk/detailed/restaurants/lastochka/']
metro : ['Воробьевы горы (1.98 км)']
lon : [ 37.54584286]
lat : [ 55.72636461]
address : ['Москва, Лужнецкая наб., причал «Лужники  ...'
 'Москва, Лужнецкая наб., причал «Лужники Северный» ...']
phone : ['+7 (495) 778-89-94']
rating : [ 4.]
timing : ['12:00-00:00']
review_count : [16]
review_link : ['/msk/opinions/restaurants/lastochka/']
avrCheck : ['2000-3000р']
description : [ 'Двухпалубный теплоход-ресторан «Ласточка», уникальныйстоличный проект, где вы можете насладиться итальянской и средиземноморскойкухней, любуясь непревзойденным видом пейзажей набережной Москвы-реки. Гостеприимство и комфорт в «Ласточке» удачно сочетаются спростой, понятной, но при этом изысканной кухней. Над меню ресторана трудятся 2шеф-повара Доменико Филиппоне и Санг Кеун. Здесь вы можете попробовать блюдаевропейской, 

In [80]:
catalog['phone'] = catalog.phone.apply(lambda x: x.replace('  ','').strip())

In [3]:
catalog['check'] = catalog.address.apply(lambda x: '...' in str(x))

In [4]:
catalog.check.value_counts()

False    1094
True      212
Name: check, dtype: int64

In [5]:
reParse = catalog[catalog.check == True]

In [6]:
def getAddress(link):
        #t = randint(1, 3)
        #print('Delay %s s.' % (t))
        #sleep(t)
        page = requests.get('https://www.restoran.ru' + link)
        data = html.fromstring(page.text)
        try:
            address = data.xpath('//div[@itemprop="address"]')[0].text_content().strip()
        except:
            address = '-'
        return address

In [7]:
%%time
res = {}
for link in reParse.link:
    res[link] = getAddress(link)

CPU times: user 15.9 s, sys: 430 ms, total: 16.4 s
Wall time: 5min 3s


In [8]:
res

{'/msk/detailed/restaurants/akvarel/': 'Адрес:\n\nул. Пришвина, д. 26, ТЦ «Миллион Мелочей», 2 этаж',
 '/msk/detailed/restaurants/alma-bar/': 'Адрес:\n\nПогодинская ул., д. 4, ЖК «Grand Deluxe»',
 '/msk/detailed/restaurants/alyaska-bar/': 'Адрес:\n\nМосковская обл., Люберцы, Октябрьский пр., д. 198',
 '/msk/detailed/restaurants/angelo/': 'Адрес:\n\nПроспект 60-летия октября, д. 5, корп. 3',
 '/msk/detailed/restaurants/aromatnaya_reka/': 'Адрес:\n\nнаб. Академика Туполева, д. 15, корп. 2',
 '/msk/detailed/restaurants/art-kafe-royal/': 'Адрес:\n\nМосковская обл., Мытищи, 3-я Крестьянская ул., д. 25',
 '/msk/detailed/restaurants/aviator-rest/': 'Адрес:\n\n                                            Пресненская наб., д. 12, Башня Федерация: Запад, 52 этаж',
 '/msk/detailed/restaurants/avshar-club/': 'Адрес:\n\n                                            Московская обл., Красногорск, Ново-Никольская ул., д. 2А',
 '/msk/detailed/restaurants/aziya/': 'Адрес:\n\nМосковская обл., Красногорск, М

In [100]:
catalog.address.value_counts()

-                                                           17
Московская обл., Красногорск, Международная ул., ...         4
Москва, ул. Петровка, д. 30/7                                4
Москва, Ленинский пр., д. 37                                 3
Москва, Кутузовский пр., д. 2/1, стр. 6                      3
Москва, ул. Кузнецкий Мост, д. 7                             3
Москва, ул. Ильинка, д. 4                                    3
Москва, ул. Арбат, д. 6/2                                    3
Москва, Тверская ул., д. 7                                   3
Москва, Пятницкая ул., д. 29                                 3
Москва, Комсомольский пр., д. 28                             2
Москва, ул. Декабристов, д. 15, стр. 2                       2
Москва, Кутузовский пр., д. 12, стр. 1                       2
Москва, Каширское шоссе, д. 80                               2
Москва, Большой Путинковский пер., д. 5                      2
Москва, Тамбовская ул., д. 4                           

- Добавить чистые адреса
- Почистить телефоны
- Добавить описания
- Добавить изображения
- Спарсить отзывы
- Описать все действия

In [10]:
r = res
for item in res:
    r[item] = res[item].replace('Адрес:','').replace('\n','').strip()

In [12]:
data_1 = catalog[catalog.check == False].copy()

In [15]:
reParse['address'] = reParse.link.apply(lambda x: r.get(x, '-'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
data = data_1.append(reParse)

In [19]:
del data['check']

In [21]:
data['phone'] = data.phone.apply(lambda x: x.replace('  ','').strip())

In [22]:
data = data.drop_duplicates()

In [27]:
img = pd.read_csv('backup.csv')

In [30]:
img = img.drop_duplicates()

In [33]:
out = data.merge(img, on='link')

In [34]:
out.to_csv('MskRestDataImgDscr.csv', index=False, encoding='utf-8')

In [89]:
out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1300 entries, 0 to 1299
Data columns (total 18 columns):
ID              1300 non-null int64
title           1300 non-null object
premium         1300 non-null bool
type            1293 non-null object
kitchen         1300 non-null object
link            1300 non-null object
metro           1300 non-null object
lon             1290 non-null float64
lat             1290 non-null float64
address         1300 non-null object
phone           1300 non-null object
rating          1300 non-null float64
timing          1300 non-null object
review_count    1300 non-null int64
review_link     1300 non-null object
avrCheck        1300 non-null object
description     1300 non-null object
image           1300 non-null object
dtypes: bool(1), float64(3), int64(2), object(12)
memory usage: 184.1+ KB


### Часть 3: Отзывы

- review_link
- author_name
- time
- rating
- text
- commentID

Не глубже 12 страницы

In [6]:
class Reviews():
    def __init__(self, catalog):
        self.data = catalog
        self.res_data = pd.DataFrame()
        
    def getRevList(self, link):
        l = (self.data[self.data.review_link == link].review_count.tolist())[0]
        res = []
        page_count = (l//20)+1
        
        for page_n in range(1, page_count+1):
            print('-----------PAGE %s PARSING-----------' % (page_n))
            try:
                page = requests.get('https://www.restoran.ru'+link+'?PAGEN_1={}'.format(page_n))
            except:
                return res
            data = html.fromstring(page.text)
            rev_list = data.xpath('//div[@class="reviews-list"]/div[@class="item"]')
            lrev = len(rev_list)
            res.extend(rev_list)
            print('-----------%s reviews add to list. ListLen %s-----------' % (lrev, len(res)))
        
        return res
        '''page_n = 1
        res = []
        lrev = 20
        while page_n < 40 and lrev == 20:
            print('-----------PAGE %s PARSING-----------' % (page_n))
            try:
                page = requests.get('https://www.restoran.ru'+link+'?PAGEN_1={}'.format(page_n))
            except:
                return res
            data = html.fromstring(page.text)
            rev_list = data.xpath('//div[@class="reviews-list"]/div[@class="item"]')
            lrev = len(rev_list)
            res.extend(rev_list)
            print('-----------%s reviews add to list. ListLen %s-----------' % (lrev, len(res)))
            page_n += 1
        return res'''
    
    def getReview(self, item):
        try:
            div = item.cssselect('div.author')[0]
        except:
            return ('-','-','-','-','-')

        try:
            name = div.cssselect('div.date a')[0].text_content().replace('\n','').strip()
        except:
            name = '-'

        try:
            if name != '-':
                date = div.cssselect('div.date')[0].text_content().replace('\n','').split(name)[1][1:].strip()
            else:
                date = div.cssselect('div.date')[0].text_content().replace('\n','').strip()
        except:
            date = '-'

        try:
            rating = len(div.cssselect('div.ratio span.icon-star'))
        except:
            rating = 0

        try:
            text = item.cssselect('div[class^="preview_text"]')[0].text_content().replace('\n','').strip()
        except:
            return ('-','-','-','-','-')

        try:
            commentID = item.cssselect('div[class^="preview_text"]')[0].attrib.get('class').replace('preview_text','')
        except:
            return ('-','-','-','-','-')

        return (name, date, rating, text, commentID)
    
    def getFrame(self, review_link, g):
        dct = {}
        #review_link = '/msk/opinions/restaurants/sixty/'
        for x,item in enumerate(g):
            dt = self.getReview(item)
            dct[x] = {
                'rLink': review_link,
                'commentID': dt[4],
                'Auhor': dt[0],
                'Date': dt[1],
                'Text': dt[3],
                'Childs': '',
                'Plus': dt[2],
                'Minus': 0,
            }
        return pd.DataFrame.from_dict(dct, orient='index')
    
    def run(self):
        clear = self.data[self.data.review_count > 1].copy()
        for link in tqdm(clear.review_link):
            if link != '-':
                print('-----------PARSING REV FROM %s PAGE-----------' % (link))
                g = self.getRevList(link)
                d = self.getFrame(link, g)
                self.res_data = self.res_data.append(d)
                print('-----------REVIEWS SHAPE: %s @ %s | RES DATA SHAPE: %s @ %s-----------' %(d.shape[0],d.shape[1], self.res_data.shape[0], self.res_data.shape[1]))
                print('------------------------------------------------------------------')

In [7]:
out = pd.read_csv('MskRestDataImgDscr.csv')

In [8]:
a = Reviews(out)

In [9]:
a.run()


  0%|          | 0/595 [00:00<?, ?it/s]

-----------PARSING REV FROM /msk/opinions/restaurants/elardzhi/ PAGE-----------
-----------PAGE 1 PARSING-----------
-----------20 reviews add to list. ListLen 20-----------
-----------PAGE 2 PARSING-----------
-----------20 reviews add to list. ListLen 40-----------
-----------PAGE 3 PARSING-----------



  0%|          | 1/595 [00:02<20:02,  2.03s/it]

-----------1 reviews add to list. ListLen 41-----------
-----------REVIEWS SHAPE: 41 @ 8 | RES DATA SHAPE: 41 @ 8-----------
------------------------------------------------------------------
-----------PARSING REV FROM /msk/opinions/restaurants/black_thai/ PAGE-----------
-----------PAGE 1 PARSING-----------
-----------20 reviews add to list. ListLen 20-----------
-----------PAGE 2 PARSING-----------
-----------20 reviews add to list. ListLen 40-----------
-----------PAGE 3 PARSING-----------
-----------20 reviews add to list. ListLen 60-----------
-----------PAGE 4 PARSING-----------
-----------20 reviews add to list. ListLen 80-----------
-----------PAGE 5 PARSING-----------
-----------20 reviews add to list. ListLen 100-----------
-----------PAGE 6 PARSING-----------
-----------20 reviews add to list. ListLen 120-----------
-----------PAGE 7 PARSING-----------
-----------20 reviews add to list. ListLen 140-----------
-----------PAGE 8 PARSING-----------
-----------20 reviews add to


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



...

-----------3 reviews add to list. ListLen 3-----------
-----------REVIEWS SHAPE: 3 @ 8 | RES DATA SHAPE: 12197 @ 8-----------
------------------------------------------------------------------
-----------PARSING REV FROM /msk/opinions/restaurants/shabada/ PAGE-----------
-----------PAGE 1 PARSING-----------


100%|█████████▉| 594/595 [15:52<00:01,  1.60s/it]

-----------2 reviews add to list. ListLen 2-----------
-----------REVIEWS SHAPE: 2 @ 8 | RES DATA SHAPE: 12199 @ 8-----------
------------------------------------------------------------------
-----------PARSING REV FROM /msk/opinions/restaurants/cafeyuzhnoe/ PAGE-----------
-----------PAGE 1 PARSING-----------
-----------20 reviews add to list. ListLen 20-----------
-----------PAGE 2 PARSING-----------
-----------20 reviews add to list. ListLen 40-----------
-----------PAGE 3 PARSING-----------
-----------20 reviews add to list. ListLen 60-----------
-----------PAGE 4 PARSING-----------
-----------20 reviews add to list. ListLen 80-----------
-----------PAGE 5 PARSING-----------


100%|██████████| 595/595 [15:57<00:00,  1.61s/it]

-----------7 reviews add to list. ListLen 87-----------
-----------REVIEWS SHAPE: 87 @ 8 | RES DATA SHAPE: 12286 @ 8-----------
------------------------------------------------------------------


In [11]:
reviews = a.res_data

In [15]:
reviews.Auhor.value_counts()

old.restoran.ru      1670
olgaart86              14
9296080                12
Катюша                 11
Андрей                 11
Костя 30               11
eduard_askim           11
timurlas               11
alla_kirova            10
Илья                   10
Edu                    10
Рус                     9
lansckaya777            9
kirill                  9
valera68                8
Настя                   8
Мурашка                 8
Alin123                 8
Павел                   8
natali                  8
Настя Клёпик            8
WhiteRoseAngel          8
Dim30                   8
filippov                8
Павел Земский111        8
Savushka                8
maksi                   8
Iriska                  8
Сергей Романов          7
Maryyy                  7
                     ... 
n.zinin                 1
grafniksan              1
mary_lis13              1
milamila                1
Ligaconsulting          1
president.aleshin       1
zalevskayatanya         1
marvic.06   

In [16]:
reviews = reviews[reviews.Auhor != 'old.restoran.ru'].drop_duplicates()

In [17]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10350 entries, 0 to 86
Data columns (total 8 columns):
Plus         10350 non-null int64
Date         10350 non-null object
Minus        10350 non-null int64
Childs       10350 non-null object
Auhor        10350 non-null object
Text         10350 non-null object
rLink        10350 non-null object
commentID    10350 non-null object
dtypes: int64(2), object(6)
memory usage: 727.7+ KB


In [18]:
reviews['Author'] = reviews['Auhor']
del reviews['Auhor']

In [21]:
%time reviews['companyID'] = reviews.rLink.apply(lambda x: (out[out.review_link == x].ID.tolist())[0])

In [22]:
del reviews['rLink']

In [24]:
reviews['commentID'] += 'c'

In [28]:
reviews.Text.value_counts()

Отзыв удалён ресторатором.                                                                                                                                                                                                                                                                                                                                                                                                102
                                                                                                                                                                                                                                                                                                                                                                                                                           29
Отзыв удален ресторатором                                                                                                                                                   

In [35]:
stop = ['Отзыв уделён ресторатором.', 'Отзыв удалён ресторатором.', 'Отзыв удален ресторатором', 'Отзыв удален ресторатором.', 'Отлично', 'Отлично!', 'Супер']

In [36]:
%%time
out_rev = reviews.copy()
for el in stop:
    out_rev = out_rev[out_rev.Text != el].copy()

CPU times: user 47.5 ms, sys: 0 ns, total: 47.5 ms
Wall time: 46.9 ms


In [38]:
out_rev.head()

,Plus,Date,Minus,Childs,Text,commentID,Author,companyID
0,5,"6 Февраля 2018, 15:22",0,,"Прекрасный ресторан, очень уютный, легко найти...",2775835c,tatf,1049250
1,5,"12 Января 2018, 10:37",0,,Был приятно удивлён.Было вкусно и уютно.Спасибо.,2765217c,cherkasov-ai,1049250
2,5,"4 Января 2018, 15:52",0,,"Вкусно, качественно, по-домашнему, хочется вер...",2762755c,yuliyapr,1049250
3,5,"7 Сентября 2017, 14:47",0,,"Очень понравилось. Вкусная грузинская кухня, щ...",2705240c,sergei.chaykin,1049250
4,5,"17 Июля 2017, 11:08",0,,Отмечала свой день рождения. Гости остались до...,2679874c,savitskaya_vita,1049250


In [39]:
out_rev['Minus'] = out_rev.Plus.apply(lambda x: x-5)

In [41]:
out_rev.to_csv('MskRestReviews.csv', index=False, encoding='utf-8')